In [1]:
from utils.bayesian_encoding import Bayesian_Encoding
from utils import utils
from utils import layer2
from utils.xgb_fn import*

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

from jupyterthemes import jtplot
jtplot.reset()

In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [3]:
train, test = utils.load_data()

Successfully loaded train and test data


In [4]:
encoded_train, encoded_test = Bayesian_Encoding(mode='likelihood').fit_transform(train, test)

[START ENCODING Fold 1/5]
[START ENCODING Fold 2/5]
[START ENCODING Fold 3/5]
[START ENCODING Fold 4/5]
[START ENCODING Fold 5/5]
Successfully Encoded


In [5]:
train, test = utils.clean_data(train, test)
train, test = utils.engineer_features(train, test)
train = train.drop(['id', 'target'], axis=1)
test = test.drop('id', axis=1)

Successfully cleaned train and test data
Successfully engineered train and test data


In [5]:
train = train.drop(train.columns[train.columns.str.startswith('ps_calc')], axis=1)
train = train.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                        'ps_ind_18_bin','ps_car_03_cat','ps_car_10_cat'], axis=1)

test = test.drop(test.columns[test.columns.str.startswith('ps_calc')], axis=1)
test = test.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                        'ps_ind_18_bin','ps_car_03_cat','ps_car_10_cat'], axis=1)

In [10]:
mul_top_5 = pd.DataFrame()
mul_top_5['ps_car_13ps_ind_05_cat'] = train['ps_ind_03']*train['ps_car_13']
mul_top_5['ps_ind_03ps_reg_03'] = train['ps_ind_03']*train['ps_reg_03']
mul_top_5['ps_reg_03ps_car_13'] = train['ps_reg_03']*train['ps_car_13']
mul_top_5['ps_reg_01ps_car_13'] = train['ps_reg_01']*train['ps_car_13']
mul_top_5['ps_ind_03ps_car_14'] = train['ps_ind_03']*train['ps_car_14']

In [7]:
interact_train = pd.DataFrame()
for i in train.columns:
    for j in train.columns:
        if i != j:
            interact_train[i+j] = train[i]*train[j]

In [5]:
meta = layer2.get_meta_features('train')
meta_test = layer2.get_meta_features('test')

In [13]:
mul_test = test.ps_ind_03*test.ps_reg_03

In [7]:
# X = meta
y = pd.read_csv('data/train.csv', na_values=-1).target
test_id = pd.read_csv('data/test.csv', na_values=-1).id

In [11]:
X = pd.concat([train, mul_top_5['ps_reg_03ps_car_13']], axis=1)
# X = X.drop(['ps_ind_03', 'ps_car_13', 'ps_reg_03', 'ps_ind_03', 'ps_reg_01', 'ps_car_14'], axis=1)
X_test = pd.concat([test, encoded_test], axis=1)

In [12]:
param = {'tree_method': 'gpu_hist',
        'silent': True,
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        'eta': 0.01,
        'max_depth': 5,
        'gamma': 0.3,
        'min_child_weight': 7,
        'subsample': 0.7,
        'colsample_bytree': 0.6,
        'max_delta_step': 2
        }

In [13]:
xgb_skf_train(X, y, param, nfolds=5, model_path_name='xgb_models/xgb')

Training xgb Fold:1/5
[0]	train-auc:0.600289	test-auc:0.597855	train-gini:0.200943	test-gini:0.195395
Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.632192	test-auc:0.626853	train-gini:0.264383	test-gini:0.253706
[200]	train-auc:0.635375	test-auc:0.628037	train-gini:0.270749	test-gini:0.256074
[300]	train-auc:0.640021	test-auc:0.63079	train-gini:0.280042	test-gini:0.261581
[400]	train-auc:0.646623	test-auc:0.634222	train-gini:0.293245	test-gini:0.268443
[500]	train-auc:0.65279	test-auc:0.637201	train-gini:0.30558	test-gini:0.274403
[600]	train-auc:0.658117	test-auc:0.639027	train-gini:0.316233	test-gini:0.278054
[700]	train-auc:0.66235	test-auc:0.640279	train-gini:0.324699	test-gini:0.280559
[800]	train-auc:0.666168	test-auc:0.641502	train-gini:0.332336	test-gini:0.283003
[900]	train-auc:0.66943	test-auc:0.642169	train-gini:0.338861	test-gini:0.284339
[1000]	train-auc:0.67

[1500]	train-auc:0.685685	test-auc:0.63831	train-gini:0.37137	test-gini:0.276619
[1600]	train-auc:0.687923	test-auc:0.638461	train-gini:0.375847	test-gini:0.276922
[1700]	train-auc:0.689863	test-auc:0.638422	train-gini:0.379726	test-gini:0.276844
Stopping. Best iteration:
[1627]	train-auc:0.688457	test-auc:0.638532	train-gini:0.376914	test-gini:0.277063

[Fold 5/5 Gini score: 0.277063]
[Successfully trained, Average Gini score: 0.2857094]


In [17]:
pred = xgb_skf_predict(X_test.values, test_id)

In [18]:
pred['target'] = pred['target']/5

In [19]:
pred.to_csv('../sub/sub43.csv', index=False)